In [2]:
import cv2
import math
import time
import carla
import torch
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
from torchvision import transforms
from model.model_culane import get_model
from stable_baselines3.common.vec_env import DummyVecEnv

ModuleNotFoundError: No module named 'torch'

In [ ]:
class CarlaEnvPPO(gym.Env):
    def __init__(self):
        super(CarlaEnvPPO, self).__init__()
        
        # Carla Connection
        self.client_ = carla.Client("localhost", 2000)
        self.client_.set_timeout(10.0)
        self.world_ = self.client.get_world()
        self.map_ = self.world_.get_map()
        
        # Spawn Point and start point
        self.spawn_points_ = self.map_.get_spawn_points()
        self.start_point_ = self.spawn_points_[0]
        
        # Vehicle Setup
        self.blueprint_library_ = self.world_.get_blueprint_library()
        self.vehicle_bp = self.blueprint_library_.find('vehicle.volkswagen.t2_2021')
        self.vehicle_ = None

        # Camera Setup
        self.camera_bp_ = self.blueprint_library_.find('camera.sensor.rgb')
        self.camera_bp_.set_attribute('image_size_x', '800')
        self.camera_bp_.set_attribute('image_size_y', '288')
        self.camera_bp_.set_attribute('fov', '90')
        self.camera_sensor = None
        
        self.camera_transform = carla.Transform(carla.Location(z=2.5, x=0.65))
    
        self.image = None
        
        # Action Space
        self.action_space = gym.spaces.Box(
            low=np.array([-1.0, -1.0]),   # steer, throttle/brake min values
            high=np.array([1.0, 1.0]),    # steer, throttle/brake max values
            dtype=np.float32
        )
        
        # Observation_space
        self.observation_space = gym.spaces.Box(
            low=0, high=255,
            shape=(288, 800, 3),
            dtype=np.uint8
        )

    
    def image_callback(self, image):
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = np.reshape(array, (image.height, image.width, 4))[:, :, :3]
        self.image = array
    
    
    def destroy_actors(self):
        if self.vehicle_ is not None:
            self.vehicle_.destroy()
        if hasattr(self, 'camera_sensor') and self.camera_sensor is not None:
            self.camera_sensor.stop()
            self.camera_sensor.destroy()

            
    def reset(self):
        self.destroy_actors()
        self.vehicle_ = self.world_.try_spawn_actor(self.vehicle_bp, self.start_point_)
        self.camera_sensor = self.world_.spawn_actor(
            self.camera_bp_,
            self.camera_transform, 
            attach_to=self.vehicle_
        )
        self.image = None
        self.camera_sensor.listen(lambda data: self.image_callback(data))

        while self.image is None:
            time.sleep(0.01)

        return self.image 
    
    def step(self, action):
        steer = action[0]
        throttle = 0
        brake = 0
        if action[1] > 0:
            throttle = action[1]
        else:
            brake = abs(action[1])
        
        self.vehicle_.apply_control(
            carla.VehicleControl(
                throttle=throttle, 
                steer=steer, 
                brake=brake
            )
        )
        
        obs = self.image
        reward = self.reward()
        done = False
        info = {}
        
        return obs, reward, done, info
    
    def reward(self):
    # Example dummy reward — replace with actual logic
        return 1.0


In [ ]:
# Vehicle Setup
# vehicle = world.get_actors().filter('*vehicle*')[0]
vehicle_bp = world.ge
start_point = spawn_points[0]

In [ ]:
# Lane Detect
import torch


In [14]:
action_space = gym.spaces.Box(
    low=np.array([-1.0, -0.5]),   # steer, throttle/brake min values
    high=np.array([1.0, 0.5]),    # steer, throttle/brake max values
    dtype=np.float32
)
for _ in range(5):
    action = action_space.sample()
    # print(action)
print(action_space.sample())

[-0.30524457  0.21000373]
